In [ ]:
import pandas as pd
import sys
# sys.path.append('../data/')
# sys.path.append('../view/')
sys.path.append('../analysis/')
from nfh_utils import *

import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget

In [ ]:
datafiles=[
    "calorische_uurwaarden_202311.csv", 
    "calorische_uurwaarden_202312.csv",
    "calorische_uurwaarden_202401.csv",
    "calorische_uurwaarden_202402.csv",
    "calorische_uurwaarden_202403.csv",
    "calorische_uurwaarden_202404.csv"
]


In [ ]:
# Function to read and process each file
def read_and_process_csv(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    return df

In [ ]:
%%time
# Read each CSV file into a DataFrame and store them in a list
dataframes = [read_and_process_csv(file) for file in datafiles]

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)

# Parse the `date_time` column as UTC
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%dT%H:%M:%S%z', utc=True)

# Convert the datetime from UTC to Europe/Amsterdam timezone
df['date_time'] = df['date_time'].dt.tz_convert('Europe/Amsterdam')

# Rename the `value` column
df.rename(columns={'value': 'gas__hhv__MJ_m_3'}, inplace=True)

# Rename the `date_time` column
df.rename(columns={'date_time': 'timestamp'}, inplace=True)    


In [ ]:
df

In [ ]:
# Calculate variability across grid_area
grid_area_variability = df.groupby('grid_area')['gas__hhv__MJ_m_3'].std()

# Calculate variability over time (across all grid_areas)
time_variability = df.groupby('timestamp')['gas__hhv__MJ_m_3'].std()

In [ ]:
grid_area_variability.mean()

In [ ]:
time_variability.mean()

In [ ]:
df_nl = df.groupby('timestamp')['gas__hhv__MJ_m_3'].mean().reset_index().set_index('timestamp')

In [ ]:
df_nl

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_nl.index, df_nl['gas__hhv__MJ_m_3'], label='Gas HHV', marker='.')
plt.xlabel('Timestamp')
plt.ylabel('Gas HHV (MJ/m^3)')
plt.title('Average calorific value of gas delivered')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_nl['gas__hhv__MJ_m_3'].describe()

In [ ]:
df_nl['gas__hhv__MJ_m_3'].max()/df_nl['gas__hhv__MJ_m_3'].mean()

In [ ]:
df_nl['gas__hhv__MJ_m_3'].mean()/df_nl['gas__hhv__MJ_m_3'].min()

In [ ]:
df_nl['gas__hhv__MJ_m_3'].max()/df_nl['gas__hhv__MJ_m_3'].min()

In [ ]:
print(f"{gas_groningen_nl_avg_hhv__J_m_3 / 1e6 :.2f}e6")

In [ ]:
df_nl['gas__hhv__MJ_m_3'].mean()*1e6/gas_groningen_nl_avg_hhv__J_m_3

In [ ]:
print(f"{temp_gas_ref__K / temp_gas_std__K : .2f}")

In [ ]:
print(f"{gas_g25_3_hhv__J__m3 * temp_gas_ref__K / temp_gas_std__K / 1e6 :.2f}e6")

In [ ]:
print(f"{gas_g20_hhv__J_m_3 * temp_gas_ref__K / temp_gas_std__K / 1e6 :.2f}e6")